In [1]:
import os, datetime
import pandas as pd
import numpy as np
import seaborn as sn
import copy
import tqdm
import talib

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
# from torch.utils.data import Dataset, DataLoader

# from sklearn import tree, svm
# from sklearn.pipeline import make_pipeline
# from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = .5
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (16, 10)
plt.rcParams['font.family'] = 'Microsoft YaHei'

In [2]:
%matplotlib qt

In [3]:
from lutils.fin.data_loader import load

In [3]:
# TQ_ROOT = 'Z:/tq_data/ticks'
# CTP_ROOT = 'Y:/fin_data'

In [4]:
# def load_ctp(exchange, symbol, day='2023-05-01'):
    
#     data_path = os.path.join(CTP_ROOT, day)
#     dp = os.path.join(data_path, '%s.%s.h5' % (exchange, symbol))
#     if not os.path.exists(dp): # and datetime.datetime.strptime(dp, '%Y-%m-%d').weekday() <:
#         return # break
#     print('load %s' % dp)

#     df = pd.read_hdf(dp)

#     # 大连商品交易所 夜盘 ActionDay 为下一个交易日, 修改为当前日期。 夜盘未过0点......
#     # 郑州商品交易所 夜盘 TradingDay 为当前交易日。 夜盘未过0点......
#     if exchange == 'DCE': 
#         df = df[(df['UpdateTime'] >= '09:00:00') & (df['UpdateTime'] <= '23:00:00')] # 非9-23点 存在难处理数据
#         df.loc[((df['UpdateTime'] > '15:15:00') & (df['UpdateTime'] <= '23:59:59')), 'ActionDay'] = df[df['UpdateTime'] >= '09:00:00'].iloc[0]['ActionDay']


#     if df.shape[0] > 0:
#         df = df[['Volume', 'Turnover', 'LastPrice', 'TradingDay', 'ActionDay', 'UpdateTime', 'UpdateMillisec', 'OpenInterest', 
#         'BidPrice1', 'BidVolume1',
#         'AskPrice1', 'AskVolume1',
#         ]]

#         df['datetime'] = df['ActionDay'] + ' ' + df['UpdateTime'] + '.' + df['UpdateMillisec'].astype(str)
        
#         df = df[['datetime', 'LastPrice', 'Volume', 'Turnover', 'OpenInterest', 'BidPrice1', 'BidVolume1', 'AskPrice1', 'AskVolume1',]].rename(columns={'datetime': 'datetime', 
#             'LastPrice': 'last_price', 
#             'Volume': 'volume', 
#             'Turnover': 'amount', 
#             'OpenInterest': 'open_interest',
#             'BidPrice1': 'bid_price1', 
#             'BidVolume1': 'bid_volume1', 
#             'AskPrice1': 'ask_price1', 
#             'AskVolume1': 'ask_volume1', 
#             }) # 'tradetime': 'tradetime'

#         df['datetime'] = pd.to_datetime(df.datetime, format='%Y%m%d %H:%M:%S.%f')
#         return df
#     else:
#         return None

In [4]:
def pingjia(price, day, rule='1S'):
    target = target_df.loc[day]
    call = call_df.loc[day]
    put = put_df.loc[day]
    
    between_times = [['09:00', '10:15'], ['10:30', '11:30'], ['13:30', '15:00'], ['21:00', '23:00']]
    
    targets = []
    calls = []
    puts = []
    for (start, end) in between_times:
        _df_target = target.between_time(start, end)
        _df_call = call.between_time(start, end)
        _df_put = put.between_time(start, end)
        
        _df_target = _df_target.last_price.resample(rule).last().ffill()
        _df_call = _df_call.last_price.resample(rule).last().ffill()
        _df_put = _df_put.last_price.resample(rule).last().ffill()
        
        targets.append(_df_target)
        calls.append(_df_call)
        puts.append(_df_put)
    
    target = pd.concat(targets).to_frame().rename(columns={'last_price': 'target'})
    call = pd.concat(calls).to_frame().rename(columns={'last_price': 'call'})
    put = pd.concat(puts).to_frame().rename(columns={'last_price': 'put'})
    
    df = pd.concat([target, call, put], axis=1).dropna()
#     print(df)
    df['price'] = price
    
    df['price1'] = df.price + df.call - df.put
    df['price2'] = df.call - df.put
    df['price3'] = df.price + df.call - .5 - df.put - .5
    df['price4'] = - df.call + df.put
    
    return df

In [5]:
def pj_range(target_df, call_df, put_df, price, start_day, end_day, rule='1S'):
    target = target_df.loc[start_day:end_day]
    call = call_df.loc[start_day:end_day]
    put = put_df.loc[start_day:end_day]
    
    between_times = [['09:00', '10:15'], ['10:30', '11:30'], ['13:30', '15:00'], ['21:00', '23:00']]
    
    targets = []
    calls = []
    puts = []
    for d in np.unique(target.index.date):
        _target = target[target.index.date == d]
        _call = call[call.index.date == d]
        _put = put[put.index.date == d]
        
        for (start, end) in between_times:
            _df_target = _target.between_time(start, end)
            _df_call = _call.between_time(start, end)
            _df_put = _put.between_time(start, end)
            
            if _df_target.size > 0:
                _df_target = _df_target.resample(rule).last().ffill()[['last_price', 'bid_price1', 'ask_price1']]
                _df_call = _df_call.resample(rule).last().ffill()[['last_price', 'bid_price1', 'ask_price1']]
                _df_put = _df_put.resample(rule).last().ffill()[['last_price', 'bid_price1', 'ask_price1']]

                targets.append(_df_target)
                calls.append(_df_call)
                puts.append(_df_put)
    
    target = pd.concat(targets).rename(columns={'last_price': 'target'})
    call = pd.concat(calls).rename(columns={'last_price': 'call', 'bid_price1': 'call_bid_price1', 'ask_price1':'call_ask_price1'})
    put = pd.concat(puts).rename(columns={'last_price': 'put', 'bid_price1': 'put_bid_price1', 'ask_price1':'put_ask_price1'})
    
#     print(target)
#     print(call)
#     print(put)
    
    df = pd.concat([target, call, put], axis=1).dropna()
    
#     df = target.join(call, how='left')
#     print(df.shape)
#     df = df.join(put, how='left')
#     print(df.shape)
#     df = pd.concat([target, call], axis=1).dropna()
#     print(df.shape)
#     df = pd.concat([df, put], axis=1).dropna()
#     print(df.shape)
    
    df['price'] = price
    
    df['price1'] = df.price + df.call - df.put
    df['price2'] = df.call - df.put
    df['price3'] = df.price + df.call - .5 - df.put - .5
    df['price4'] = - df.call + df.put
    
    df['price5'] = df.price + df.call_bid_price1 - df.put_ask_price1
#     df[df['price5'] > 100000].price5 = np.nan
    
    pm = df.price1.rolling('5T')
    df['price1_mean'] = pm.mean()
    df['price1_std'] = pm.std()
    
    return df

In [7]:
# price = 4500

# exchange = 'SHFE'
# target_symbol = 'rb2305'
# call_symbol = '%sC%s' % (target_symbol, price)
# put_symbol = '%sP%s' % (target_symbol, price)

In [5]:
# price = 3900

# exchange = 'SHFE'
# target_symbol = 'rb2308'
# call_symbol = '%sC%s' % (target_symbol, price)
# put_symbol = '%sP%s' % (target_symbol, price)

In [8]:
# target_df = load(exchange, target_symbol, '2023-07-10')
# call_df = load(exchange, call_symbol, '2023-07-10')
# put_df = load(exchange, put_symbol, '2023-07-10')

load Y:/ctp_data\2023-07-11\SHFE.rb2308.h5
load Y:/ctp_data\2023-07-12\SHFE.rb2308.h5
load Y:/ctp_data\2023-07-13\SHFE.rb2308.h5
load Y:/ctp_data\2023-07-14\SHFE.rb2308.h5
load Y:/ctp_data\2023-07-11\SHFE.rb2308C3900.h5
load Y:/ctp_data\2023-07-12\SHFE.rb2308C3900.h5
load Y:/ctp_data\2023-07-13\SHFE.rb2308C3900.h5
load Y:/ctp_data\2023-07-14\SHFE.rb2308C3900.h5
load Y:/ctp_data\2023-07-11\SHFE.rb2308P3900.h5
load Y:/ctp_data\2023-07-12\SHFE.rb2308P3900.h5
load Y:/ctp_data\2023-07-13\SHFE.rb2308P3900.h5
load Y:/ctp_data\2023-07-14\SHFE.rb2308P3900.h5


In [6]:
price = 3950

exchange = 'SHFE'
target_symbol = 'rb2310'
call_symbol = '%sC%s' % (target_symbol, price)
put_symbol = '%sP%s' % (target_symbol, price)
start_date = '2023-08-01'

In [7]:
target_df = load(exchange, target_symbol, start_date)
call_df = load(exchange, call_symbol, start_date)
put_df = load(exchange, put_symbol, start_date)

load Y:/ctp_data\2023-08-02\SHFE.rb2310.h5
load Y:/ctp_data\2023-08-03\SHFE.rb2310.h5
load Y:/ctp_data\2023-08-04\SHFE.rb2310.h5
load Y:/ctp_data\2023-08-02\SHFE.rb2310C3950.h5
load Y:/ctp_data\2023-08-03\SHFE.rb2310C3950.h5
load Y:/ctp_data\2023-08-04\SHFE.rb2310C3950.h5
load Y:/ctp_data\2023-08-02\SHFE.rb2310P3950.h5
load Y:/ctp_data\2023-08-03\SHFE.rb2310P3950.h5
load Y:/ctp_data\2023-08-04\SHFE.rb2310P3950.h5


In [8]:
# target_df

In [9]:
# target_df.index = target_df.datetime
# call_df.index = call_df.datetime
# put_df.index = put_df.datetime

In [10]:
# day = '2023-06-13'

In [11]:
# _df = pingjia(price, day)

In [12]:
# start_day = '2023-03-10'
# end_day = '2023-06-16'
# start_day = '2023-07-10'
# end_day = '2023-07-16'
start_day = '2023-08-02'
end_day = '2023-08-04'

In [13]:
_df = pj_range(target_df, call_df, put_df, price, start_day, end_day)

In [14]:
_df2 = _df

In [15]:
_df2 = _df2.drop(_df[np.abs(_df.price5) > 1000000].index)

In [16]:
# _df2.price5

In [17]:
# plt.plot(d.target.values, label='标的价格')
# plt.plot(d.price.values, label='合成标的现价')

# plt.legend()

def plot_price(df):
    fig, ax1 = plt.subplots()

    ax1.plot(df.target.values, c='r', label='标的价格')
    ax1.plot(df.price1.values, c='b', label='合成标的现价')
    # ax1.plot(df.price1_mean.values, c='deepskyblue', label='合成标的现价均值')
    # ax1.plot(df.price1_std.values, c='aqua', label='合成标的现价标准差')
    # ax1.plot(df.price3.values, c='y', label='合成标的现价3')

    ax1.plot(df.price5.values, c='g', label='正向复合头寸 买价卖价')

    # ax1.get_xaxis().get_major_formatter().set_scientific(False)

    ax2 = ax1.twinx()
    # ax2.plot(df.price2.values, c='lime', label='正向复合头寸')
    # ax2.plot(df.price4.values, c='gold', label='反向复合头寸')


    # ax2.plot(df.price1_std.values, c='aqua', label='合成标的现价标准差')

    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc=0)

    # ax2.get_xaxis().get_major_formatter().set_scientific(False)
    ax2.format_coord = lambda x,y: f"x={np.int32(x)}, y={np.format_float_positional(y)}"

    plt.title('%s %s %s %s-%s' % (target_symbol, call_symbol, put_symbol, start_day, end_day))

    plt.show()

In [18]:
# list(_df2[(np.abs(_df2.price5 - _df2.target) > 5) & (np.abs(_df2.price5 - _df2.target) < 10)].index) # [['target','price1', 'price5']]

In [19]:
plot_price(_df2.loc['2023-08-02 21'])

In [97]:
# _df2.loc['2023-08-02 09']

In [23]:
_df.iloc[34188]

target             3808.000000
bid_price1         3808.000000
ask_price1         3809.000000
call                117.000000
call_bid_price1     111.500000
call_ask_price1     120.000000
put                 107.500000
put_bid_price1       83.000000
put_ask_price1      129.500000
price              3800.000000
price1             3809.500000
price2                9.500000
price3             3808.500000
price4               -9.500000
price5             3782.000000
price1_mean        3801.471154
price1_std            2.751850
Name: 2023-07-24 21:00:51, dtype: float64

In [50]:
_df # .loc['2023-07-12'] # 2023-03-20, 2023-03-21, 2023-03-22

,target,bid_price1,ask_price1,call,call_bid_price1,call_ask_price1,put,put_bid_price1,put_ask_price1,price,price1,price2,price3,price4,price5,price1_mean,price1_std
datetime,,,,,,,,,,,,,,,,,
2023-07-11 13:30:00,3681.0,3675.0,3688.0,5.0,4.0,6.0,232.0,94.5,2.465000e+02,3900,3673.0,-227.0,3642.5,227.0,3.657500e+03,3673.000000,NaN
2023-07-11 13:30:01,3681.0,3679.0,3685.0,5.0,4.0,6.0,232.0,102.5,1.797693e+308,3900,3673.0,-227.0,3642.5,227.0,-1.797693e+308,3673.000000,0.000000
2023-07-11 13:30:02,3681.0,3679.0,3684.0,5.0,4.0,6.0,232.0,102.5,1.797693e+308,3900,3673.0,-227.0,3642.5,227.0,-1.797693e+308,3673.000000,0.000000
2023-07-11 13:30:03,3681.0,3680.0,3684.0,5.0,4.0,6.0,232.0,102.5,1.797693e+308,3900,3673.0,-227.0,3642.5,227.0,-1.797693e+308,3673.000000,0.000000
2023-07-11 13:30:04,3681.0,3680.0,3684.0,5.0,4.0,6.0,232.0,102.5,1.797693e+308,3900,3673.0,-227.0,3642.5,227.0,-1.797693e+308,3673.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-14 22:59:48,3720.0,3721.0,3728.0,3.0,3.0,3.5,174.5,145.0,5.770000e+02,3900,3728.5,-171.5,3698.0,171.5,3.326000e+03,3733.683333,12.745439
2023-07-14 22:59:49,3720.0,3721.0,3728.0,3.0,3.0,3.5,174.5,145.0,5.770000e+02,3900,3728.5,-171.5,3698.0,171.5,3.326000e+03,3733.581667,12.664206
2023-07-14 22:59:50,3720.0,3711.0,3728.0,3.0,3.0,3.5,174.5,145.0,5.770000e+02,3900,3728.5,-171.5,3698.0,171.5,3.326000e+03,3733.480000,12.581624


In [30]:
_df.iloc[122524]

target         4224.000000
call             34.500000
put             300.500000
price          4500.000000
price1         4234.000000
price2         -266.000000
price3         4203.500000
price4          266.000000
price1_mean    4234.208333
price1_std        0.246915
Name: 2023-03-20 10:39:31, dtype: float64

In [14]:
%matplotlib inline

In [15]:
# call_df.index.date[0].strftime('%Y-%m-%d')

In [16]:
# r = os.path.join('D:/option/xxxxx', str(price))
# if not os.path.exists(r): os.makedirs(r)
# for d in np.unique(call_df.index.date):
#     day = d.strftime('%Y-%m-%d')
    
#     _df = pingjia(price, day)
    
#     plt.plot(_df.target.values, label='标的价格')
#     plt.plot(_df.price.values, label='合成标的现价')
#     plt.legend()
    
#     plt.savefig(os.path.join(r, '%s.png' % day))
#     plt.close()
#     print(day)

In [17]:
exchange = 'SHFE'

In [18]:
targets = ['rb2306'] # 'rb2304', 'rb2305', 'rb2306', 'rb2307', 'rb2308'

In [19]:
prices = [3300, 3400, 3500, 3600, 3700, 3800, 3900, 4000, 4100, 4200, 4300]

In [20]:
import gc

In [21]:
gc.collect()

173

In [22]:
# tl_root = os.path.join('D:/option/tl1')
# for t in targets:
#     gc.collect()
#     target_df = load(exchange, t)
#     if target_df is None:
#         continue
#     target_df.index = target_df.datetime
#     for price in prices:
#         call_symbol = '%sC%s' % (t, price)
#         put_symbol = '%sP%s' % (t, price)
#         call_df = load(exchange, call_symbol)
#         put_df = load(exchange, put_symbol)
        
#         if call_df is None or put_df is None:
#             continue
        
#         call_df.index = call_df.datetime
#         put_df.index = put_df.datetime
        
#         r = os.path.join(tl_root, t, str(price))
#         if not os.path.exists(r): os.makedirs(r)
#         for d in np.unique(call_df.index.date):
#             day = d.strftime('%Y-%m-%d')

#             _df = pingjia(price, day)

#             plt.plot(_df.target.values, label='标的价格')
#             plt.plot(_df.price.values, label='合成标的现价')
            
#             plt.title('%s %s %s %s' % (t, call_symbol, put_symbol, day))
#             plt.legend()
            
#             plt.savefig(os.path.join(r, '%s.png' % day))
#             plt.close()
#             print(day)
    
#     del target_df
#     del call_df
#     del put_df
    
#     gc.collect()

In [23]:
tl_root = os.path.join('D:/option/tl2')
for t in targets:
    gc.collect()
    target_df = load(exchange, t)
    if target_df is None:
        continue
    target_df.index = target_df.datetime
    for price in prices:
        call_symbol = '%sC%s' % (t, price)
        put_symbol = '%sP%s' % (t, price)
        call_df = load(exchange, call_symbol)
        put_df = load(exchange, put_symbol)
        
        if call_df is None or put_df is None:
            continue
        
        call_df.index = call_df.datetime
        put_df.index = put_df.datetime
        
        r = os.path.join(tl_root, t, str(price))
        if not os.path.exists(r): os.makedirs(r)
        for d in np.unique(call_df.index.date):
            day = d.strftime('%Y-%m-%d')

            _df = pingjia(price, day)
            
            fig, ax1 = plt.subplots()

            ax1.plot(_df.target.values, c='r', label='标的价格')
            ax1.plot(_df.price.values, c='b', label='合成标的现价')

            ax2 = ax1.twinx()
            ax2.plot(_df.price2.values, c='g', label='合成标的现价2')

            lines, labels = ax1.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            ax2.legend(lines + lines2, labels + labels2, loc=0)
            
            plt.title('%s %s %s %s' % (t, call_symbol, put_symbol, day))
            
            plt.savefig(os.path.join(r, '%s.png' % day))
            plt.close()
            print(day)
    
    del target_df
    del call_df
    del put_df
    
    gc.collect()

load Z:/tq_data/ticks\SHFE.rb2306.h5
load Y:/fin_data\2023-05-04\SHFE.rb2306.h5
load Y:/fin_data\2023-05-05\SHFE.rb2306.h5
load Y:/fin_data\2023-05-08\SHFE.rb2306.h5
load Y:/fin_data\2023-05-09\SHFE.rb2306.h5
load Y:/fin_data\2023-05-10\SHFE.rb2306.h5
load Y:/fin_data\2023-05-11\SHFE.rb2306.h5
load Y:/fin_data\2023-05-12\SHFE.rb2306.h5
load Y:/fin_data\2023-05-15\SHFE.rb2306.h5
load Y:/fin_data\2023-05-16\SHFE.rb2306.h5
load Y:/fin_data\2023-05-17\SHFE.rb2306.h5
load Y:/fin_data\2023-05-18\SHFE.rb2306.h5
load Y:/fin_data\2023-05-19\SHFE.rb2306.h5
load Y:/fin_data\2023-05-22\SHFE.rb2306.h5
load Y:/fin_data\2023-05-23\SHFE.rb2306.h5
load Y:/fin_data\2023-05-24\SHFE.rb2306.h5
load Y:/fin_data\2023-05-25\SHFE.rb2306.h5
load Y:/fin_data\2023-05-26\SHFE.rb2306.h5
load Y:/fin_data\2023-05-29\SHFE.rb2306.h5
load Y:/fin_data\2023-05-30\SHFE.rb2306.h5
load Y:/fin_data\2023-05-31\SHFE.rb2306.h5
load Y:/fin_data\2023-06-01\SHFE.rb2306.h5
load Y:/fin_data\2023-06-02\SHFE.rb2306.h5
load Y:/fin_data\

load Y:/fin_data\2023-05-24\SHFE.rb2306P3500.h5
load Y:/fin_data\2023-05-25\SHFE.rb2306P3500.h5
2022-12-28
2022-12-29
2022-12-30
2023-01-03
2023-01-04
2023-01-05
2023-01-06
2023-01-09
2023-01-10
2023-01-11
2023-01-12
2023-01-13
2023-01-16
2023-01-17
2023-01-18
2023-01-19
2023-01-20
2023-01-30
2023-01-31
2023-02-01
2023-02-02
2023-02-03
2023-02-06
2023-02-07
2023-02-08
2023-02-09
2023-02-10
2023-02-13
2023-02-14
2023-02-15
2023-02-16
2023-02-17
2023-02-20
2023-02-21
2023-02-22
2023-02-23
2023-02-24
2023-02-27
2023-02-28
2023-03-01
2023-03-02
2023-03-03
2023-03-06
2023-03-07
2023-03-08
2023-03-09
2023-03-10
2023-03-13
2023-03-14
2023-03-15
2023-03-16
2023-03-17
2023-03-20
2023-03-21
2023-03-22
2023-03-23
2023-03-24
2023-03-27
2023-03-28
2023-03-29
2023-03-30
2023-03-31
2023-04-03
2023-04-04
2023-04-06
2023-04-07
2023-04-10
2023-04-11
2023-04-12
2023-04-13
2023-04-14
2023-04-17
2023-04-18
2023-04-19
2023-04-20
2023-04-21
2023-04-24
2023-04-25
2023-04-26
2023-04-27
2023-04-28
2023-05-04
20

2023-01-04
2023-01-05
2023-01-06
2023-01-09
2023-01-10
2023-01-11
2023-01-12
2023-01-13
2023-01-16
2023-01-17
2023-01-18
2023-01-19
2023-01-20
2023-01-30
2023-01-31
2023-02-01
2023-02-02
2023-02-03
2023-02-06
2023-02-07
2023-02-08
2023-02-09
2023-02-10
2023-02-13
2023-02-14
2023-02-15
2023-02-16
2023-02-17
2023-02-20
2023-02-21
2023-02-22
2023-02-23
2023-02-24
2023-02-27
2023-02-28
2023-03-01
2023-03-02
2023-03-03
2023-03-06
2023-03-07
2023-03-08
2023-03-09
2023-03-10
2023-03-13
2023-03-14
2023-03-15
2023-03-16
2023-03-17
2023-03-20
2023-03-21
2023-03-22
2023-03-23
2023-03-24
2023-03-27
2023-03-28
2023-03-29
2023-03-30
2023-03-31
2023-04-03
2023-04-04
2023-04-06
2023-04-07
2023-04-10
2023-04-11
2023-04-12
2023-04-13
2023-04-14
2023-04-17
2023-04-18
2023-04-19
2023-04-20
2023-04-21
2023-04-24
2023-04-25
2023-04-26
2023-04-27
2023-04-28
2023-05-04
2023-05-05
2023-05-08
2023-05-09
2023-05-10
2023-05-11
2023-05-12
2023-05-15
2023-05-16
2023-05-17
2023-05-18
2023-05-19
2023-05-22
2023-05-23

2023-01-20
2023-01-30
2023-01-31
2023-02-01
2023-02-02
2023-02-03
2023-02-06
2023-02-07
2023-02-08
2023-02-09
2023-02-10
2023-02-13
2023-02-14
2023-02-15
2023-02-16
2023-02-17
2023-02-20
2023-02-21
2023-02-22
2023-02-23
2023-02-24
2023-02-27
2023-02-28
2023-03-01
2023-03-02
2023-03-03
2023-03-06
2023-03-07
2023-03-08
2023-03-09
2023-03-10
2023-03-13
2023-03-14
2023-03-15
2023-03-16
2023-03-17
2023-03-20
2023-03-21
2023-03-22
2023-03-23
2023-03-24
2023-03-27
2023-03-28
2023-03-29
2023-03-30
2023-03-31
2023-04-03
2023-04-04
2023-04-06
2023-04-07
2023-04-10
2023-04-11
2023-04-12
2023-04-13
2023-04-14
2023-04-17
2023-04-18
2023-04-19
2023-04-20
2023-04-21
2023-04-24
2023-04-25
2023-04-26
2023-04-27
2023-04-28
2023-05-04
2023-05-05
2023-05-08
2023-05-09
2023-05-10
2023-05-11
2023-05-12
2023-05-15
2023-05-16
2023-05-17
2023-05-18
2023-05-19
2023-05-22
2023-05-23
2023-05-24
2023-05-25
load Z:/tq_data/ticks\SHFE.rb2306C4200.h5
load Y:/fin_data\2023-05-04\SHFE.rb2306C4200.h5
load Y:/fin_data\20

In [24]:
# ax2.legend(lines + lines2, labels + labels2, loc=0)
            
# plt.title('%s %s %s %s' % (t, call_symbol, put_symbol, day))

# plt.savefig(os.path.join(r, '%s.png' % day))
# plt.close()
# print(day)

In [134]:
rb2308 = load('SHFE', 'rb2308', '2023-07-10')
rb2309 = load('SHFE', 'rb2309', '2023-07-10')
rb2310 = load('SHFE', 'rb2310', '2023-07-10')

rb2308C3800 = load('SHFE', 'rb2308C3800', '2023-07-10')
rb2309C3800 = load('SHFE', 'rb2309C3800', '2023-07-10')
rb2310C3800 = load('SHFE', 'rb2310C3800', '2023-07-10')

load Y:/ctp_data\2023-07-11\SHFE.rb2308.h5
load Y:/ctp_data\2023-07-12\SHFE.rb2308.h5
load Y:/ctp_data\2023-07-13\SHFE.rb2308.h5
load Y:/ctp_data\2023-07-14\SHFE.rb2308.h5
load Y:/ctp_data\2023-07-11\SHFE.rb2309.h5
load Y:/ctp_data\2023-07-12\SHFE.rb2309.h5
load Y:/ctp_data\2023-07-13\SHFE.rb2309.h5
load Y:/ctp_data\2023-07-14\SHFE.rb2309.h5
load Y:/ctp_data\2023-07-11\SHFE.rb2310.h5
load Y:/ctp_data\2023-07-12\SHFE.rb2310.h5
load Y:/ctp_data\2023-07-13\SHFE.rb2310.h5
load Y:/ctp_data\2023-07-14\SHFE.rb2310.h5
load Y:/ctp_data\2023-07-11\SHFE.rb2308C3800.h5
load Y:/ctp_data\2023-07-12\SHFE.rb2308C3800.h5
load Y:/ctp_data\2023-07-13\SHFE.rb2308C3800.h5
load Y:/ctp_data\2023-07-14\SHFE.rb2308C3800.h5
load Y:/ctp_data\2023-07-11\SHFE.rb2309C3800.h5
load Y:/ctp_data\2023-07-12\SHFE.rb2309C3800.h5
load Y:/ctp_data\2023-07-13\SHFE.rb2309C3800.h5
load Y:/ctp_data\2023-07-14\SHFE.rb2309C3800.h5
load Y:/ctp_data\2023-07-11\SHFE.rb2310C3800.h5
load Y:/ctp_data\2023-07-12\SHFE.rb2310C3800.h5
load

In [137]:
a = rb2309.last_price - rb2308.last_price
b = rb2309C3800.last_price - rb2308C3800.last_price

c = rb2310.last_price - rb2308.last_price
d = rb2310C3800.last_price - rb2308C3800.last_price

In [141]:
plt.plot(c, label='期货')
plt.plot(d, label='期权', marker='.')
plt.legend()
plt.show()

In [139]:
c

datetime
2023-07-10 23:00:00.500    15.0
2023-07-10 23:00:00.500    15.0
2023-07-10 23:00:00.500    15.0
2023-07-10 23:00:00.500    15.0
2023-07-11 08:59:00.500    14.0
                           ... 
2023-07-14 22:59:58.500    15.0
2023-07-14 22:59:59.000     NaN
2023-07-14 22:59:59.500     NaN
2023-07-14 23:00:00.000     NaN
2023-07-14 23:00:00.500    15.0
Name: last_price, Length: 165662, dtype: float64

In [140]:
d

datetime
2023-07-11 11:30:00.500    65.0
2023-07-11 13:30:00.500    65.0
2023-07-11 13:30:01.000    65.0
2023-07-11 13:30:01.500     NaN
2023-07-11 13:30:10.500     NaN
                           ... 
2023-07-14 22:59:55.500     NaN
2023-07-14 22:59:56.000     NaN
2023-07-14 22:59:57.500     NaN
2023-07-14 22:59:58.000     NaN
2023-07-14 23:00:00.500    77.5
Name: last_price, Length: 77457, dtype: float64

In [148]:
rb2309C3800[rb2309C3800.ask_price1 > 9999999].bid_price1

datetime
2023-07-11 18:37:17.500    1.797693e+308
2023-07-12 18:36:29.500    1.797693e+308
2023-07-12 20:59:00.500     1.500000e+00
2023-07-12 21:00:00.500     1.500000e+00
2023-07-12 21:00:13.000     2.500000e+00
                               ...      
2023-07-14 21:03:12.500     1.950000e+01
2023-07-14 21:05:44.500     1.950000e+01
2023-07-14 22:57:03.000     1.500000e+00
2023-07-14 22:58:39.000     1.500000e+00
2023-07-14 23:00:00.500     1.500000e+00
Name: bid_price1, Length: 133, dtype: float64

In [150]:
b = rb2308C3800.ask_price1 - rb2308C3800.bid_price1

In [161]:
plt.plot(b, marker='.')